In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import random

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Set the prop id lookup field dictionary

In [101]:
#Define the dict of dicts that contains the feature classes and the translated column names
field_lookup = {'property_evw': {'propnum': 'gispropnum',
                                 'prop id': 'gispropnum', 
                                 'borough': 'department',
                                 'ampsdistrict': 'department',
                                 'prop name': 'signname',
                                 'site name': 'signname',
                                 'prop location': 'location',
                                 'site location': 'location',
                                 'acres': 'acres', 
                                 'jurisdiction': 'jurisdiction', 
                                 'typecategory': 'typecategory', 
                                 'featurestatus':'featurestatus', 
                                 'gisobjid':'gisobjid'}, 
                 'playground_evw': {'propnum': 'parentid', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'signname', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},  
                 'zone_evw': {'propnum': 'parentid', 
                              'prop id': 'omppropid', 
                              'borough': 'department',
                              'ampsdistrict': 'department',
                              'site name': 'sitename', 
                              'site location': 'location', 
                              'acres': 'acres', 
                              'jurisdiction': 'jurisdiction', 
                              'featurestatus':'featurestatus', 
                              'gisobjid':'gisobjid'}, 

                 'unmapped_gisallsites_evw': {'propnum': 'gispropnum',
                                              'prop id': 'omppropid', 
                                              'borough': 'department',
                                              'ampsdistrict': 'department',
                                              'site name': 'name',
                                              'site location': 'location',
                                              'acres': 'acres', 
                                              'jurisdiction': 'jurisdiction'},
                 'schoolyard_to_playground_evw': {'propnum': 'gispropnum',
                                                  'prop id': 'gispropnum', 
                                                  'borough': 'department',
                                                  'ampsdistrict': 'department',
                                                  'prop name': 'signname',
                                                  'site name': 'signname',
                                                  'prop location': 'location',
                                                  'site location': 'location',
                                                  'acres': 'acres', 
                                                  'jurisdiction': 'jurisdiction', 
                                                  'featurestatus':'featurestatus'},
                 'greenstreet_evw': {'propnum': 'gispropnum', 
                                     'prop id': 'omppropid', 
                                     'borough': 'department',
                                     'ampsdistrict': 'department',
                                     'site name': 'sitename', 
                                     'site location': 'location', 
                                     'acres': 'acres', 
                                     'jurisdiction': 'jurisdiction', 
                                     'featurestatus':'featurestatus', 
                                     'gisobjid':'gisobjid'},
                 'golfcourse_evw': {'propnum': 'gispropnum', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'name', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},
                 'restrictivedeclarationsite_evw': {'propnum': 'gispropnum',
                                                    'prop id': 'gispropnum', 
                                                    'borough': 'department',
                                                    'ampsdistrict': 'department',
                                                    'prop name': 'signname',
                                                    'site name': 'signname',
                                                    'prop location': 'location',
                                                    'site location': 'location',
                                                    'acres': 'acres', 
                                                    'jurisdiction': 'jurisdiction', 
                                                    'featurestatus':'featurestatus'}, 
                 'structure_evw': {'propnum': 'gispropnum',
                                   'prop id': 'omppropid', 
                                   'borough': 'department',
                                   'ampsdistrict': 'department',
                                   'prop name': 'description',
                                   'site name': 'description',
                                   'prop location': 'location',
                                   'site location': 'location',
                                   'jurisdiction': 'jurisdiction',  
                                   'featurestatus':'featurestatus', 
                                   'gisobjid':'gisobjid'}}

In [14]:
#Define a dict of source feature classes that map to the sourcefc value
sourcefc_lookup = {'property_evw': 'Property', 
                   'playground_evw': 'Playground', 
                   'zone_evw': 'Zone', 
                   'unmapped_gisallsites_evw': 'Unmapped',
                   'schoolyard_to_playground_evw': 'Schoolyard To Playground',
                   'greenstreet_evw': 'Greenstreet',
                   'golfcourse_evw': 'GolfCourse',
                   'restrictivedeclarationsite_evw': 'RestrictiveDeclarationSite',
                   'structure_evw': 'Structure'}

In [52]:
field_lookup_as = {f: ['['+ i[1] + '] as ' + '['+ i[0] + ']' for i in field_lookup[f].items()] for f in sourcefc_lookup }

## Original GIS Data

In [54]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw', 'structure_evw']

In [55]:
#Create the list of SQL Queries
gis_sql_list = ["select objectid, {}, '{}' as sourcefc from parksgis.dpr.{}"
                .format(' ,'.join(field_lookup_as[t]), sourcefc_lookup[t], t) 
                        for t in gis_tables]

In [56]:
#Create a dictionary with sources and dataframes that contain the original data
gis_df_list = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(gis_sql_list, gis_tables)}

## Original PIP Data

In [47]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_compatible_inspected_sites', 'allsites']

In [48]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [49]:
#Create a list of dataframes with the original dat
pip_df_list = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

## Perform DML Steps

### Delete Records

In [88]:
#Define the string to use in the where clause
where_str = '13'

In [89]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} like '%" + where_str + "'%'").format(field_lookup[f]['prop id']) for f in field_lookup}

In [91]:
#Filter DFs based on the same where string
gis_df_pre_delete = {g: gis_df_list[g][gis_df_list[g]['prop id'].str.contains(where_str) == True] for g in gis_tables}

In [93]:
#Loop through the GIS datasets, drop the _evw and create the queries that perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where[g]) for g in gis_tables]

In [16]:
gis_con = gis_engine.connect()

In [18]:
#Loop through and execute the delete queries
for q in sql:
    gis_con.execute(q)

In [ ]:
#Create a list of dataframes with the data after deletes, notably all dfs should be empty
gis_df_post_delete = {g: gis_df_list[g][gis_df_list[g]['prop id'].str.contains(where_str) == True] for g in gis_tables}

### Update Records

In [112]:
#Define the string to use in the where clause
where_str = '14'

In [114]:
#Define the common where clause using the prop id field lookup for each source GIS feature class
where = {f: str(r"where {} like '%" + where_str + "'%'").format(field_lookup[f]['prop id']) for f in field_lookup}

In [115]:
#Filter DFs based on the same where string
gis_df_pre_update = {g: gis_df_list[g][gis_df_list[g]['prop id'].str.contains(where_str) == True] for g in gis_tables}

In [117]:
set_value = {g: str(r"[{}] = 'Testing Updates'").format(field_lookup[g]['site location']) for g in gis_tables}

In [120]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['update parksgis.dpr.{} set {}  {}'.format(g.replace('_evw', ''), set_value[g], where[g]) for g in gis_tables]

In [121]:
#Loop through and execute the update queries
for q in sql:
    gis_con.execute(q)

["update parksgis.dpr.property set [location] = 'Testing Updates'  where gispropnum like '%14'%'",
 "update parksgis.dpr.playground set [location] = 'Testing Updates'  where omppropid like '%14'%'",
 "update parksgis.dpr.zone set [location] = 'Testing Updates'  where omppropid like '%14'%'",
 "update parksgis.dpr.unmapped_gisallsites set [location] = 'Testing Updates'  where omppropid like '%14'%'",
 "update parksgis.dpr.schoolyard_to_playground set [location] = 'Testing Updates'  where gispropnum like '%14'%'",
 "update parksgis.dpr.greenstreet set [location] = 'Testing Updates'  where omppropid like '%14'%'",
 "update parksgis.dpr.golfcourse set [location] = 'Testing Updates'  where omppropid like '%14'%'",
 "update parksgis.dpr.restrictivedeclarationsite set [location] = 'Testing Updates'  where gispropnum like '%14'%'",
 "update parksgis.dpr.structure set [location] = 'Testing Updates'  where omppropid like '%14'%'"]

### Insert Non-Duplicate Records

In [130]:
#Define the range for generating records
n_gen = range(0, 100)

In [143]:
gis_list

NameError: name 'gis_list' is not defined

In [131]:
#Define the borough portion of IDs
boro = ['B','Q','M','R','X'] 

In [149]:
#Combine the borough portion with the formated (padded with leading 0) random numbers
nums = [{gis_tables[random.randint(0, len(gis_tables) - 1)]: boro[random.randint(0,4)] + f'{random.randint(0, 999):03}'} for i in n_gen]

## PICK UP HERE!

In [173]:
for n in nums:
    for g in gis_tables:
        print({g:n[g] if })

SyntaxError: invalid syntax (<ipython-input-173-37332f018a19>, line 3)

In [211]:
test = []

In [212]:
for g in gis_tables:
    for n in nums:
        if g in n:
            print(n[g])
            test.append(n[g])
            print(test)

B386
['B386']
R620
['B386', 'R620']
Q550
['B386', 'R620', 'Q550']
X212
['B386', 'R620', 'Q550', 'X212']
R358
['B386', 'R620', 'Q550', 'X212', 'R358']
X197
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197']
Q794
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794']
X730
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730']
M541
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541']
R520
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541', 'R520']
R662
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541', 'R520', 'R662']
Q571
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541', 'R520', 'R662', 'Q571']
M973
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541', 'R520', 'R662', 'Q571', 'M973']
B779
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541', 'R520', 'R662', 'Q571', 'M973', 'B779']
B049
['B386', 'R620', 'Q550', 'X212', 'R358', 'X197', 'Q794', 'X730', 'M541'

In [183]:
[[n[g]] for n in nums for g in gis_tables if g in n]

[['B798'],
 ['B386'],
 ['B779'],
 ['M463'],
 ['Q804'],
 ['B039'],
 ['M738'],
 ['Q474'],
 ['R620'],
 ['Q187'],
 ['M853'],
 ['M655'],
 ['X469'],
 ['X274'],
 ['R720'],
 ['Q550'],
 ['Q706'],
 ['X212'],
 ['Q293'],
 ['R628'],
 ['Q676'],
 ['X519'],
 ['R358'],
 ['Q056'],
 ['X027'],
 ['B049'],
 ['Q964'],
 ['X586'],
 ['M607'],
 ['X197'],
 ['Q261'],
 ['Q906'],
 ['B978'],
 ['B129'],
 ['Q794'],
 ['R809'],
 ['Q003'],
 ['X278'],
 ['B690'],
 ['R424'],
 ['Q670'],
 ['X231'],
 ['Q301'],
 ['R439'],
 ['M512'],
 ['Q628'],
 ['X730'],
 ['M286'],
 ['Q867'],
 ['M541'],
 ['M946'],
 ['R474'],
 ['R090'],
 ['Q860'],
 ['R743'],
 ['R883'],
 ['Q203'],
 ['Q559'],
 ['R520'],
 ['Q467'],
 ['R891'],
 ['B677'],
 ['R643'],
 ['Q650'],
 ['M150'],
 ['Q950'],
 ['R225'],
 ['R662'],
 ['Q173'],
 ['R430'],
 ['Q613'],
 ['Q553'],
 ['X207'],
 ['Q571'],
 ['X525'],
 ['B079'],
 ['X306'],
 ['M095'],
 ['M885'],
 ['B637'],
 ['X408'],
 ['X369'],
 ['M728'],
 ['R080'],
 ['B354'],
 ['Q063'],
 ['M614'],
 ['B706'],
 ['Q664'],
 ['X587'],
 ['R089'],

In [156]:
for n in nums:
    if n.keys == 'property_evw':
        print(n)

In [133]:
nums

['M286',
 'B421',
 'B024',
 'B323',
 'Q113',
 'B392',
 'R301',
 'B628',
 'R640',
 'B977',
 'Q236',
 'B437',
 'M418',
 'M919',
 'Q874',
 'R117',
 'B069',
 'M406',
 'M800',
 'B497',
 'B709',
 'X407',
 'B561',
 'R751',
 'M372',
 'X204',
 'B228',
 'Q269',
 'R054',
 'R269',
 'R551',
 'Q537',
 'X665',
 'R714',
 'Q502',
 'Q854',
 'Q735',
 'X961',
 'Q301',
 'Q928',
 'B092',
 'Q161',
 'Q174',
 'Q939',
 'M711',
 'R659',
 'M149',
 'R196',
 'X717',
 'Q789',
 'B609',
 'Q342',
 'X871',
 'M972',
 'R614',
 'R635',
 'M075',
 'Q140',
 'B787',
 'X849',
 'B115',
 'B417',
 'R727',
 'B384',
 'Q310',
 'X583',
 'X049',
 'B950',
 'Q382',
 'X800',
 'R118',
 'B950',
 'R139',
 'R071',
 'B315',
 'X206',
 'M321',
 'M633',
 'Q094',
 'B798',
 'B146',
 'M874',
 'M612',
 'R901',
 'Q170',
 'R695',
 'X865',
 'M796',
 'M354',
 'M219',
 'B231',
 'M685',
 'M269',
 'Q531',
 'X987',
 'X182',
 'B350',
 'X285',
 'X170',
 'M759']

In [126]:
gis_df_copy = {g: gis_df_list[g] for g in gis_tables}
    

In [128]:
gis_df_copy['property_evw']

,objectid,propnum,prop id,borough,ampsdistrict,prop name,site name,prop location,site location,acres,jurisdiction,typecategory,featurestatus,gisobjid,sourcefc
0,5255,B456,B456,B-08,B-08,Walt L Shemal Garden,Walt L Shemal Garden,Dean St. between Bedford Ave. and Franklin Ave.,Dean St. between Bedford Ave. and Franklin Ave.,0.149,DPR,Garden,Active,100003698.0,Property
1,6102,B155,B155,B-05,B-05,Triangle,Triangle,"New Lots Ave., Dumont Ave., Atkins Ave.","New Lots Ave., Dumont Ave., Atkins Ave.",0.017,DPR,Triangle/Plaza,Active,100004026.0,Property
2,4949,M257,M257,M-04,M-04,Clement Clarke Moore Park,Clement Clarke Moore Park,W. 22 St. and 10 Ave.,W. 22 St. and 10 Ave.,0.489,DPR,Neighborhood Park,Active,100003727.0,Property
3,5851,M212,M212,M-01,M-01,The Queen Elizabeth II September 11th Garden,The Queen Elizabeth II September 11th Garden,"Hanover Sq., Pearl St. and Stone St.","Hanover Sq., Pearl St. and Stone St.",0.117,DPR,Triangle/Plaza,Active,100004584.0,Property
4,5146,B271,B271,B-17,B-17,Railroad Playground,Railroad Playground,Ditmas Ave. bet. E. 92 St. and E. 91 St.,Ditmas Ave. bet. E. 92 St. and E. 91 St.,2.300,DPR,Neighborhood Park,Active,100004239.0,Property
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174,5701,X001,X001,X-07,X-07,Aqueduct Walk,Aqueduct Walk,Aqueduct Ave. W. bet. W. Fordham Rd. and W. Ki...,Aqueduct Ave. W. bet. W. Fordham Rd. and W. Ki...,4.926,DPR,Community Park,Active,100005096.0,Property
2175,5455,B077,B077,B-06,B-06,Coffey Park,Coffey Park,Verona St. bet. Richard St. and Dwight St.,Verona St. bet. Richard St. and Dwight St.,8.271,DPR,Neighborhood Park,Active,100004314.0,Property
2176,4638,X172,X172,X-11,X-11,Allerton Playground,Allerton Playground,Allerton Ave. bet. Throop Ave. and Bouck Ave.,Allerton Ave. bet. Throop Ave. and Bouck Ave.,1.597,DPR/DOE,Jointly Operated Playground,Active,100004422.0,Property
2177,5977,M339,M339,M-03,M-03,El Sol Brilliante Jr,El Sol Brilliante Jr,E. 12 St. bet. Ave. A and Ave. B,E. 12 St. bet. Ave. A and Ave. B,0.059,DPR,Garden,Active,100003976.0,Property


### Insert Duplicate Records